In [2]:
#https://rueki.tistory.com/26

import pandas as pd

df = pd.read_excel('C:/Users/09368/workspace/data/excel/Article_economy_202101_202104.xlsx')

print(len(df))

title  = df['title'].values.tolist()
body   = df['body'].values.tolist()
date   = df['date'].values.tolist()
source = df['source'].values.tolist()

18472


## 전처리

In [16]:
import re

def preprocessing(sentence):
    sentence =re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', sentence)
    return sentence

# Preprocessing (특수문자 제거)
df['body_cleaned'] = df['body'].apply(preprocessing)
df['overview']     = df['body_cleaned'].fillna(' ') # 키워드 중심 추출 위해

overview = df['overview'].values.tolist()

## tf-idf 계산

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(overview)

# 본문에 대해서 tf-idf 수행
# 기사 각각의 tf-idf 값을 계산
print(tfidf_matrix.shape)

# TF-IDF 매트릭스 확인
# 데이터 갯수 : 1000, 모든 본문 뉴스를 토대로 구한 유니크한 단어의 갯수 : 56230
# 1000개 데이터의 overview에서 56230개의 단어가 쓰인 것을 볼 수 있다.

(18472, 292731)


## 코사인 유사도 계산

#### 뉴스 데이터의 본문 통해 유사도를 비교, 뉴스 title로 비슷한 뉴스 찾기

In [2]:
"""
코사인 유사도 : 두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도

두 벡터의 방향이 같으면 1, 직교일 때는 0, 서로 반대 방향이면 -1
1에 가까울 수록 유사도가 높다고 판단

ex) 
문서1 : 저는 사과 좋아요
문서2 : 저는 바나나 좋아요
문서3 : 저는 바나나 좋아요 저는 바나나 좋아요

        바나나   사과   저는    좋아요
문서1   0         1       1        1
문서2   1         0       1        1
문서3   2         0       2        2 

"""

from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
    return dot(A,B) / (norm(A)*norm(B))

doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

print(cos_sim(doc1,doc2))
print(cos_sim(doc2,doc3))
print(cos_sim(doc1,doc3))

0.6666666666666667
1.0000000000000002
0.6666666666666667


In [17]:
from sklearn.metrics.pairwise import linear_kernel

# 코사인 유사도 계산
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#인덱스 테이블 생성
indices = pd.Series(df[:1000].index, index= df['title'][:1000]).drop_duplicates()

print(cosine_sim)
"""
각 행과 열은 들어온 본문 데이터이며, 각 셀은 코사인 유사도 수치를 의미함
"""
print(indices.head())

[[1.         0.03528597 0.03388586 ... 0.00608131 0.00205572 0.00119064]
 [0.03528597 1.         0.02907445 ... 0.         0.00290602 0.00187008]
 [0.03388586 0.02907445 1.         ... 0.00378803 0.00672677 0.01048857]
 ...
 [0.00608131 0.         0.00378803 ... 1.         0.01133439 0.00545765]
 [0.00205572 0.00290602 0.00672677 ... 0.01133439 1.         0.3869048 ]
 [0.00119064 0.00187008 0.01048857 ... 0.00545765 0.3869048  1.        ]]
title
세진중공업 560억 규모 육상플랜트 모듈 수주           0
한국조선해양 지난해 수주액 100억 달러…목표 91% 달성    1
“경기 하락세 1분기에는 다소 완화될 듯”             2
울산 소상공인 81%는 ‘임차인’…보증금 네 번째 높아      3
공영쇼핑 ‘2021 건강하소 행복하소’ 이벤트           4
dtype: int64


In [12]:
idx= indices['한국조선해양 지난해 수주액 100억 달러…목표 91% 달성']
print(idx)

1


In [32]:
def get_recommendatioin(title, cosine_sim=cosine_sim):
    # 입려된 뉴스 title로부터 인덱스 가져오기
    idx= indices[title]
    
    #모든 영화에 대해 해당 영화와의 유사도
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    #유사도 따라 영화들 정렬
    sim_scores = sorted(sim_scores,key=lambda x:x[1], reverse=True)
    
    #유사한 뉴스 10개
    sim_scores = sim_scores[1:11] 
    
    #10개의 뉴스 인덱스 받아온다
    news_indices = [i[0] for i in sim_scores]
    
    # 기존에 읽어들인 데이터에서 해당 인덱스의 값 가져옴. 스코어 열 추가(코사인 유사도 확인)
    result = df.iloc[news_indices].copy()
    result['score'] = [i[1] for i in sim_scores]
    
    del result['body']
    
    df_result = result[['title','score']]
    
    return df_result # df['title'].iloc[news_indices] #10개의 뉴스 제목 리턴


get_recommendatioin('한국조선해양 지난해 수주액 100억 달러…목표 91% 달성')

,title,score
6048,현대重 그룹 올해 수주 목표 16조원… 작년의 1.5배 규모,0.219697
9352,한국조선해양 올해 첫 수주…컨테이너선 6척,0.177582
3525,한국조선해양 올해 수주 목표 149억 달러로 설정,0.172858
13449,한국조선해양 올해 수주 목표 149억 달러로 설정,0.172858
12525,특징주한국조선해양 올해 목표 수주액 상향에 5% 급등,0.160713
4457,한국조선해양 올해 수주 목표 149억 달러 설정…작년 1.5배,0.157546
5293,한국조선해양 올해 수주 목표 149억 달러로 설정,0.143414
11184,한국조선해양 새해 벽두부터 대형 컨테이너선 6척 9000억 수주,0.136390
10519,조선업 새해에도 수주 낭보...올해 수주목표 높인다,0.135477
9705,새해부터 ‘수주릴레이’ 조선사… 실적 갱신 기대,0.134530


In [31]:
# test
get_recommendatioin('공영쇼핑 ‘2021 건강하소 행복하소’ 이벤트')

,title,score
741,공영쇼핑 2021 건강하소 행복하소 이벤트,0.699871
888,비대면 수요에 보복소비까지…신축년 기대하는 홈쇼핑 이색 이벤트 풍성,0.396550
7554,출첵하고 가소…BYC 온라인 신년 이벤트 풍성,0.168408
5476,롯데百 새해맞이 비대면 쇼핑 이벤트 연다,0.166348
893,이커머스 업체들 2021년 맞아 소·새해 행사 진행,0.165688
5774,롯데百 온라인몰서 겨울 상품 30% 할인…소프라이즈 이벤트,0.164705
6747,BYC 2021년 새해 맞이 이벤트 실시,0.158350
6310,롯데하이마트온라인쇼핑몰 새해맞이 1월 ‘빅하트세일’ 실시,0.143038
12908,마켓컬리 설 선물 얼리버드 최대 60% 할인,0.133724
12849,삼성물산 패션삼성전자 31일까지 공동 프로모션,0.128659
